# Here we are going to do ETL process step by step

## 1.) Extract the Data

In [1]:
# We have the .csv file which we want to inport

import pandas as pd
import numpy as np

crypto = pd.read_csv("crypto-markets.csv")
crypto.head()

,slug,asset,name,date,ranknow,open,high,low,close,volume,market,close_ratio,spread
0,target-coin,TGT,Target Coin,29-09-2017,607,0.028961,0.054766,0.028961,0.041777,69996,0.0,0.4966,0.03
1,target-coin,TGT,Target Coin,30-09-2017,607,0.041783,0.046196,0.031435,0.031744,5725,0.0,0.0209,0.01
2,target-coin,TGT,Target Coin,01-10-2017,607,0.031761,0.035957,0.021040,0.028385,5012,0.0,0.4924,0.01
3,target-coin,TGT,Target Coin,02-10-2017,607,0.028375,0.054595,0.020417,0.022525,8010,0.0,0.0617,0.03
4,target-coin,TGT,Target Coin,03-10-2017,607,0.022527,0.032225,0.020211,0.020359,1787,0.0,0.0123,0.01


> We have the data price in USD currency, perhaps we would like to change it to IDR for research purpose for specific coins (ETH, BTC, AIR, LTC)

In [2]:
# current rate 1 USD = 14,430 IDR

## 2.) Transform the Data

Like usual we would like make every column desired with corresponding format of the data. And get rid of null values and remove unrelavant column.

In [3]:
crypto['asset'].unique()

array(['TGT', 'symbol', 'HGT', 'HBZ', 'GET', 'ETHOS', 'DASH', 'CBT',
       'BTC', 'AIR', 'XRP', 'LTC'], dtype=object)

Before that we would like apply the price conversion method to only (ETH, BTC, AIR, LTC) coins.

In [4]:
coin_asset = ['ETH', 'BTC', 'AIR', 'LTC']

# coverting open, close, high and low price of crypto currencies to IDR
crypto['open'] = crypto[['open', 'asset']].apply(lambda x: (x[0]*14430) if x[1] in coin_asset else np.nan, axis=1)
crypto['high'] = crypto[['high', 'asset']].apply(lambda x: (x[0]*14430) if x[1] in coin_asset else np.nan, axis=1)
crypto['low'] = crypto[['low', 'asset']].apply(lambda x: (x[0]*14430) if x[1] in coin_asset else np.nan, axis=1)
crypto['close'] = crypto[['close', 'asset']].apply(lambda x: (x[0]*14430) if x[1] in coin_asset else np.nan, axis=1)


# Remove NaN value
crypto.dropna(inplace = True)

try: 
    crypto.loc[crypto['asset']=='BTC']
except:
    print('code wrong')
    

#Reset index
crypto.reset_index(drop=True ,inplace=True)
crypto.head()


,slug,asset,name,date,ranknow,open,high,low,close,volume,market,close_ratio,spread
0,bitcoin,BTC,Bitcoin,28-04-2013,1,1952379.0,1962191.4,1906203.0,1936650.3,0,1.500520e+09,0.5438,3.88
1,bitcoin,BTC,Bitcoin,29-04-2013,1,1939969.2,2128280.7,1933620.0,2085712.2,0,1.491160e+09,0.7813,13.49
2,bitcoin,BTC,Bitcoin,30-04-2013,1,2077920.0,2120199.9,1934341.5,2005770.0,0,1.597780e+09,0.3843,12.88
3,bitcoin,BTC,Bitcoin,01-05-2013,1,2005770.0,2018612.7,1554399.6,1688165.7,0,1.542820e+09,0.2882,32.17
4,bitcoin,BTC,Bitcoin,02-05-2013,1,1679363.4,1812408.0,1331600.4,1518180.3,0,1.292190e+09,0.3881,33.32


In [5]:
# Remove unrelevant columns

crypto.drop(labels=['slug', 'ranknow', 'volume', 'market', 'close_ratio', 'spread'], inplace=True, axis=1)
crypto.head()

,asset,name,date,open,high,low,close
0,BTC,Bitcoin,28-04-2013,1952379.0,1962191.4,1906203.0,1936650.3
1,BTC,Bitcoin,29-04-2013,1939969.2,2128280.7,1933620.0,2085712.2
2,BTC,Bitcoin,30-04-2013,2077920.0,2120199.9,1934341.5,2005770.0
3,BTC,Bitcoin,01-05-2013,2005770.0,2018612.7,1554399.6,1688165.7
4,BTC,Bitcoin,02-05-2013,1679363.4,1812408.0,1331600.4,1518180.3


Our Data is done transformed, cleaned and ready to be export to __Data Visualizations (Graphs)__, __PDF__ or __Excel report__, or __database__

# 3.) Load the Data

> In this practice, I am going to load it to SQL through in-built SQL module called ‘sqlite3’ for Python3

In [6]:
import sqlite3

# connect function opens a connection to the SQLite database file, 
conn = sqlite3.connect('session.db')
print(conn)

# Drop a table name Crypto if it exists already
try:
    conn.execute('DROP TABLE IF EXISTS `Crypto` ')
except Exception as e:
    raise(e)
finally:
    print('Table dropped')

try:
    conn.execute('''
         CREATE TABLE Crypto
         (ID         INTEGER PRIMARY KEY,
         ASSET       TEXT    NOT NULL,
         NAME        TEXT    NOT NULL,
         Date        datetime,
         Open        Float DEFAULT 0,
         High        Float DEFAULT 0,
         Low         Float DEFAULT 0,
         Close       Float DEFAULT 0);''')
    print ("Table created successfully");
except Exception as e:
    print(str(e))
    print('Table Creation Failed!!!!!')
finally:
    conn.close() # this closes the database connection

Table dropped
Table created successfully


In [7]:
# this will convert pandas dataframe to list 
crypto_list = crypto.values.tolist()

# lets make new connection to Insert crypto data in SQL DB
conn = sqlite3.connect('session.db')

# make a cursor - it will help with querying SQL DB
cur = conn.cursor()

try:
    # will use ? sign to represent each column names inside VALUE().
    cur.executemany("INSERT INTO Crypto(ASSET, NAME, Date, Open, High, Low, Close) VALUES (?,?,?,?,?,?,?)", crypto_list)
    cur.execute("SELECT * FROM Crypto")
    #cur.execute(sql)
    conn.commit()
    print('Data Inserted Successfully')
    print(cur.fetchall()) #Print all data
except Exception as e:
    print(str(e))
    print('Data Insertion Failed')
finally:
    # finally block will help with always closing the connection to DB even in case of error.
    conn.close()

# Output: Data Inserted Successfully

Data Inserted Successfully
[(1, 'BTC', 'Bitcoin', '28-04-2013', 1952379.0000000002, 1962191.4, 1906203.0, 1936650.3), (2, 'BTC', 'Bitcoin', '29-04-2013', 1939969.2, 2128280.7, 1933620.0, 2085712.2), (3, 'BTC', 'Bitcoin', '30-04-2013', 2077920.0, 2120199.9, 1934341.5000000002, 2005770.0), (4, 'BTC', 'Bitcoin', '01-05-2013', 2005770.0, 2018612.6999999997, 1554399.6, 1688165.7), (5, 'BTC', 'Bitcoin', '02-05-2013', 1679363.4, 1812408.0, 1331600.4, 1518180.2999999998), (6, 'BTC', 'Bitcoin', '03-05-2013', 1533187.5, 1560315.9, 1141413.0, 1410532.5), (7, 'BTC', 'Bitcoin', '04-05-2013', 1415583.0, 1659450.0, 1334775.0, 1623375.0), (8, 'BTC', 'Bitcoin', '05-05-2013', 1629147.0, 1714284.0, 1546030.2, 1672581.3), (9, 'BTC', 'Bitcoin', '06-05-2013', 1673591.4000000001, 1798843.8, 1538815.2, 1620489.0), (10, 'BTC', 'Bitcoin', '07-05-2013', 1619767.5, 1636939.2, 1409811.0, 1608945.0), (11, 'BTC', 'Bitcoin', '08-05-2013', 1581528.0, 1670705.4, 1581528.0, 1638815.0999999999), (12, 'BTC', 'Bitcoin', '0